In [3]:
import copy
import numpy as np
import numpy.linalg as la
import time
from collections import deque
from operator import itemgetter
import sys
import queue as Q

In [65]:
class maze():
    
#   create a maze with list;
    def __init__(self, maze="mediumMaze.txt"):
        self.cost = {}
        self.raw_list = []
        self.maze_list = []
        self.width = None
        self.height = None
        self.end = [] # ready for multiple dots
        self.start = None
        self.visited = [] 
        self.solution = [] # ready for draw;
        self.expandedNumber = 0
        
        with open(maze) as f:
            for line in f:
                self.raw_list.append(line.rstrip())
        self.width = len(self.raw_list[0])
        self.height = len(self.raw_list)
        
        for line in self.raw_list:
            for char in line:
                if char == "%":
                    self.maze_list.append(0)
                elif char == ".":
                    self.maze_list.append("G")
                    self.end.append((len(self.maze_list)%self.width-1, len(self.maze_list)//self.width))
                elif char == "P":
                    self.maze_list.append("P")
                    self.start = ((len(self.maze_list)%self.width-1, len(self.maze_list)//self.width))
                else:
                    self.maze_list.append(1)
                    
        for g in self.end:
            for index, cell in enumerate(self.maze_list):
                x = index%self.width
                y = index//self.width
                if not cell:
                    self.cost[(x,y)] = np.inf
                else:
                    self.cost[(x,y)] = abs(x - g[0]) + abs(y - g[1])
                    
    def accessMaze(self, x, y):
        return self.maze_list[self.getPoint(x, y)]                
    
    def getPoint(self, x, y):
        if x < self.width and y < self.height:
            return y*self.width + x
        else:
            print("Index out of bound!")
            return False
  
    def getCordinate(self, point):
        return (point%self.width, point//self.width)
    
    def manhattanDistance(self, cur, des): return abs(cur[0] - des[0]) + abs(cur[1] - des[1])
    
#######################################################################################################################
#######################################################################################################################
       
    def adjacentList_AStar(self, pos, visited):
        adjacent = []
        x, y = pos   
        if self.accessMaze(x+1, y):
            if (x+1, y) not in visited:
                adjacent.append((x+1, y))
            else:
                if visited[pos] + 1 < visited[(x+1, y)]:
                    adjacent.append((x+1, y))  
                    
        if self.accessMaze(x, y+1):
            if (x, y+1) not in visited:
                adjacent.append((x, y+1))
            else:
                if visited[pos] + 1 < visited[(x, y+1)]:
                    adjacent.append((x, y+1))
                    
        if self.accessMaze(x-1, y):
            if (x-1, y) not in visited:
                adjacent.append((x-1, y))
            else:
                if visited[pos] + 1 < visited[(x-1, y)]:
                    adjacent.append((x-1, y))
                    
        if self.accessMaze(x, y-1):
            if (x, y-1) not in visited:
                adjacent.append((x, y-1))
            else:
                if visited[pos] + 1 < visited[(x, y-1)]:
                    adjacent.append((x, y-1))
        
        return adjacent
    
    def PQSort(self, start, goals):
        queue = Q.PriorityQueue()
        for node in goals:
            goals.remove(node)
            queue.put((self.manhattanDistance(start, node), node))
        while not queue.empty():
            current = queue.get()[1]
            goals.append(current)
        return goals
    
    def AStar(self, start):
        self.end = self.PQSort(start, self.end)
        self.AStarHelper(start)
        
#     def AStar(self, start):
#         self.expandedNumber = 0
#         queue = Q.PriorityQueue()
#         queue.put((0, start))    
#         visited = {}
#         visited[start] = 0
#         parent = {}
        
#         while not queue.empty():
#             current = queue.get()[1]
#             self.expandedNumber += 1
            
#             if current in self.end:
#                 while current != start:
#                     self.solution.append(current)
#                     current = parent[current]
#                 self.solution.append(current)
#                 return True
            
#             for node in self.adjacentList_AStar(current, visited):  
#                 parent[node] = current
#                 visited[node] = visited[current] + 1
#                 queue.put((visited[node] + self.cost[node], node))
                          
#         return False
  
    def AStarHelper(self, start):
        queue = Q.PriorityQueue()
        queue.put((0, start))    
        visited = {}
        visited[start] = 0
        parent = {}
        
        while not queue.empty() and len(self.end) != 0:
            current = queue.get()[1]
            self.expandedNumber += 1
            
            if current in self.end:
                self.solution.append(current)
                self.end.remove(current)
                while current != start:
                    self.solution.append(current)
                    current = parent[current]
                return self.AStarHelper(current)
            
            for node in self.adjacentList_AStar(current, visited):  
                parent[node] = current
                visited[node] = visited[current] + 1
                queue.put((visited[node] + self.cost[node], node))
                          
        return True

    
#######################################################################################################################
#######################################################################################################################


    def adjacentList(self, pos):
        adjacent = []
        x, y = pos
        if self.accessMaze(x+1, y) and self.getPoint(x+1, y) not in self.visited:
            adjacent.append((x+1, y))
        if self.accessMaze(x, y+1) and self.getPoint(x, y+1) not in self.visited:
            adjacent.append((x, y+1))
        if self.accessMaze(x-1, y) and self.getPoint(x-1, y) not in self.visited:
            adjacent.append((x-1, y))
        if self.accessMaze(x, y-1) and self.getPoint(x, y-1) not in self.visited:
            adjacent.append((x, y-1))
        return adjacent
    
    def bfs(self, start):
        self.bfsHelper(start)
       
    def bfsHelper(self, start):
        self.start = start
        self.visited = []
        queue = []
        parent = {}
        queue.append(start)
        self.visited.append(self.getPoint(start[0], start[1]))
        while queue and len(self.end) != 0:
            current = queue[0]
            queue = queue[1:]
            self.expandedNumber += 1
            if current in self.end:
                #print("I get to the destination!")
                self.end.remove(current)
                # The last step to the solution, track back
                self.solution.append(current)

                while current != self.start:
                    self.solution.append(current)
                    current = parent[current]
                self.solution = self.solution[::-1]

                    #new search:              
                return self.bfsHelper(current)

            adjacent_nodes = self.adjacentList(current)
            for node in adjacent_nodes:
                parent[node] = current
                queue.append(node)
                self.visited.append(self.getPoint(node[0], node[1]))     

        return True


#######################################################################################################################
#######################################################################################################################


    def drawSolution(self):
        solution = self.raw_list[:]
        for pos in self.solution:
            x, y = pos
            if self.raw_list[y][x] == ".":
                solution[y] = solution[y][:x] + "G" + solution[y][x+1:]
            elif self.raw_list[y][x] == "P":
                solution[y] = solution[y][:x] + "P" + solution[y][x+1:]
            else:
                solution[y] = solution[y][:x] + "." + solution[y][x+1:]
        return solution

In [48]:
m = maze("bigMaze.txt")
#m = maze("openMaze.txt")
%timeit m.AStar(m.start)
print(m.expandedNumber)
#%timeit m.bfs(m.start)
m.drawSolution()

23.3 µs ± 2.01 µs per loop (mean ± std. dev. of 7 runs, 10000 loops each)
1159


['%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%',
 '% %           %                       %   %     %       %   %               %..G%',
 '% % %%% %%% %%% %% %%%%%%%%%% %%%%%%% %%% % %%% % % %%% % % % %%%%% %%% %%%%%.% %',
 '%   %     %   % %   %       %   %   %   % % %   % % %     % %     % % % %.....% %',
 '% %%%%    %%% % %%% % % %%% %%% % % %%% % % % %%% % %%%%% % %     % % % %.%%%%% %',
 '%         %   %   %   % % % %   % %       %     %       % % % %   % %   %.%     %',
 '%%% %% %% % %%%%% % %%% % % %  %% % %%%% %% %%% % %%%%% % % % % %%% % %%%.% %%% %',
 '%       % % %   % % % % %   %     % %       % %   %   %   %     %   % %...% % % %',
 '% %%%%%%% % % % % % % % %%% % %%%%% % %%%%%%% %%%%%%% %%%%%%% %%% %%% %.%%% % % %',
 '%     %   %   %   % % %   %   %           %     %   %   %         %   %...% % % %',
 '%%%%% % %%%%%   %%% % %%% % %%% %%%     % % %%% % % %%% % %%%%%%% %%% %%%.% % % %',
 '%         %     %   % %   % %   %   %   %   %     % 

In [66]:
m = maze("mediumMaze.txt")
#m = maze("openMaze.txt")
%timeit m.AStar(m.start)
print(m.expandedNumber)
#%timeit m.bfs(m.start)
m.drawSolution()

23.3 µs ± 2.81 µs per loop (mean ± std. dev. of 7 runs, 10000 loops each)
519


['%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%',
 '%P..  %       %             %         %     %           %   %',
 '%%%.%%% % %%%%% %%%%% %%%%% %%% %%%%%%% % %%% % %%%%%%% % %%%',
 '%...%.....    %   % %   % %   %         %   % %     % % %   %',
 '%.%%%.%%%.%%% %%% % %%% % %%% % %%%%%% %%%% % %%%%% % % %%% %',
 '%.%...% %...% %   %   % %                 % %   % % %       %',
 '%...%%% % %.% % % % % % %%% % %%%%%%% %%% % %%% % % %%%%%%% %',
 '% %   %   %.%   %   % %   % %     %   %   %     % %   %   % %',
 '% %%%%%%% %.% %%% %%% %%% % %%% %%% % % %%%%%%%%% %%% % % % %',
 '%   %   % %...    % % %   %   % %   % % %     %     %   % % %',
 '% % %%% % %%%.%%%%% % % %%%%% % % %%% % % %%% %%%%% %%%%% %%%',
 '% %   % %   %.......  % %   % %     %   % % %           %   %',
 '% %%%%% %%%%% %%%%%.%%% % % % %%%%%%%%% % % %%% %%% %%% %%% %',
 '%     %       %    .....% % %         %   %     %     %   % %',
 '% %%% %%% %%%%% %%%%% %.%%% %%% %%%%% %%%%%% %%%% %%% %%% % %',
 '%   %   

In [60]:
m = maze("mediumMaze.txt")
#m = maze("openMaze.txt")
#%timeit m.AStar(m.start)
%timeit m.bfs(m.start)
print(m.expandedNumber)
m.drawSolution()

972 ns ± 24.6 ns per loop (mean ± std. dev. of 7 runs, 1000000 loops each)
632


['%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%',
 '%P..  %       %             %         %     %           %   %',
 '%%%.%%% % %%%%% %%%%% %%%%% %%% %%%%%%% % %%% % %%%%%%% % %%%',
 '%...%.........%   % %   % %   %         %   % %     % % %   %',
 '%.%%%.%%% %%%.%%% % %%% % %%% % %%%%%% %%%% % %%%%% % % %%% %',
 '%.%...% %   %.%   %   % %                 % %   % % %       %',
 '%...%%% % % %.% % % % % %%% % %%%%%%% %%% % %%% % % %%%%%%% %',
 '% %   %   % %.  %   % %   % %     %   %   %     % %   %   % %',
 '% %%%%%%% % %.%%% %%% %%% % %%% %%% % % %%%%%%%%% %%% % % % %',
 '%   %   % %  .    % % %   %   % %   % % %     %     %   % % %',
 '% % %%% % %%%.%%%%% % % %%%%% % % %%% % % %%% %%%%% %%%%% %%%',
 '% %   % %   %.......  % %   % %     %   % % %  .....    %   %',
 '% %%%%% %%%%% %%%%%.%%% % % % %%%%%%%%% % % %%%.%%%.%%% %%% %',
 '%     %       %    .....% % %  .......%   % ....%  ...%   % %',
 '% %%% %%% %%%%% %%%%% %.%%% %%%.%%%%%.%%%%%%.%%%% %%%.%%% % %',
 '%   %   

In [47]:
m = maze("smallSearch.txt")
%timeit m.AStar(m.start)
print(m.expandedNumber)
m.drawSolution()

0
22.1 µs ± 2.07 µs per loop (mean ± std. dev. of 7 runs, 10000 loops each)
1629


['%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%',
 '%G.....P.G     %......G..G.  %',
 '%  .%%%%%.%%%%%%.%. % % %%.  %',
 '%  .%  % .   %  .%. % %  %..G%',
 '%  ...G% ....G...%G   %  %%%%%',
 '%%%%%.%%%%.%%%  %%%%%%%      %',
 '%G....    .......G    % %%%  %',
 '%%.%%%%%%%%.%%%%%%%%%%% %G   %',
 '% .     %  .........    %.%%%%',
 '%..     %%%%%...  %G......   %',
 '%G% %%%    % .%.  %% %% %%%%%%',
 '%          % G%........G     %',
 '%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%']

In [39]:
m = maze("smallSearch.txt")
%timeit m.bfs(m.start)
print(m.expandedNumber)
m.drawSolution()

961 ns ± 26.6 ns per loop (mean ± std. dev. of 7 runs, 1000000 loops each)
1564


['%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%',
 '%G.....P.G     %......G..G...%',
 '%  .%%%%%.%%%%%%.%. % % %%  .%',
 '%  .%  % ....%  .%. % %  %  G%',
 '%  ...G%  . .G...%G   %  %%%%%',
 '%%%%%.%%%%.%%% .%%%%%%%      %',
 '%G....    ..   ..G    % %%%  %',
 '%%.%%%%%%%%.%%%%%%%%%%% %G   %',
 '% .     %  .............%.%%%%',
 '%..     %%%%%.    %G   ...   %',
 '%G% %%%    % .%   %% %%.%%%%%%',
 '%          % G%        G     %',
 '%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%']